In [44]:
import os
import re

import requests
from bs4 import BeautifulSoup
import json
from pathlib import Path
from langchain.chat_models import ChatVertexAI
from langchain.agents import AgentType, initialize_agent

from typing import Tuple
"""
from pydantic import BaseModel

from langchain.embeddings.base import Embeddings
from langchain.vectorstores.base import VectorStore
from langchain.llms import VertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.vectorstores.chroma import Chroma
from vertexai.preview.language_models import TextGenerationModel
from langchain import PromptTemplate
"""
from usp.tree import sitemap_tree_for_homepage
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np
from google.cloud import bigquery
from price_parser import Price

In [2]:
PROJECT_ID = "dastex-genai" 
REGION = "us-central1"  
EMBED_FOLDER = "embeddings_website"
JSON_FILE_NAME = "scraped_website_text.json"

In [4]:
def get_urls(websites, filter):
    """
    Scrapes websites for URLs that match the given filter.

    Args:
        websites (list): A list of website URLs to scrape.
        filter (list): A list of keywords to filter the URLs.

    Returns:
        list: A list of URLs that match the filter.
    """
    urls = []
    for website in websites:
        print(f"Scraping website {website}\n")
        tree = sitemap_tree_for_homepage(website)
        pages = [page.url for page in tree.all_pages() if any(word.lower() in page.url.lower() for word in filter)]
        # filtered_pages = filter_strings(pages, words)
        urls += pages
    return urls

In [16]:
tree = sitemap_tree_for_homepage('https://www.bvkap.de/der-bvk/mitglieder/mitglieder-details.html')

2023-09-12 10:01:50,630 WARNING usp.tree [6161/MainThread]: Assuming that the homepage of https://www.bvkap.de/der-bvk/mitglieder/mitglieder-details.html is https://www.bvkap.de/
2023-09-12 10:01:50,632 INFO usp.fetch_parse [6161/MainThread]: Fetching level 0 sitemap from https://www.bvkap.de/robots.txt...
2023-09-12 10:01:50,634 INFO usp.helpers [6161/MainThread]: Fetching URL https://www.bvkap.de/robots.txt...
2023-09-12 10:01:50,920 INFO usp.fetch_parse [6161/MainThread]: Parsing sitemap from URL https://www.bvkap.de/robots.txt...
2023-09-12 10:01:50,921 INFO usp.fetch_parse [6161/MainThread]: Fetching level 0 sitemap from https://www.bvkap.de/sitemap.xml...
2023-09-12 10:01:50,922 INFO usp.helpers [6161/MainThread]: Fetching URL https://www.bvkap.de/sitemap.xml...
2023-09-12 10:01:51,114 INFO usp.fetch_parse [6161/MainThread]: Parsing sitemap from URL https://www.bvkap.de/sitemap.xml...
2023-09-12 10:01:51,176 INFO usp.fetch_parse [6161/MainThread]: Fetching level 0 sitemap from ht

In [17]:
tree

IndexWebsiteSitemap(url=https://www.bvkap.de/, sub_sitemaps=[IndexRobotsTxtSitemap(url=https://www.bvkap.de/robots.txt, sub_sitemaps=[PagesXMLSitemap(url=https://www.bvkap.de/sitemap.xml, pages=[SitemapPage(url=https://www.bvkap.de/, priority=0.5, last_modified=None, change_frequency=None, news_story=None), SitemapPage(url=https://www.bvkap.de/politik.html, priority=0.5, last_modified=None, change_frequency=None, news_story=None), SitemapPage(url=https://www.bvkap.de/unsere-positionen.html, priority=0.5, last_modified=None, change_frequency=None, news_story=None), SitemapPage(url=https://www.bvkap.de/unsere-positionen/mittelstand.html, priority=0.5, last_modified=None, change_frequency=None, news_story=None), SitemapPage(url=https://www.bvkap.de/unsere-positionen/nachhaltigkeit.html, priority=0.5, last_modified=None, change_frequency=None, news_story=None), SitemapPage(url=https://www.bvkap.de/unsere-positionen/start-ups.html, priority=0.5, last_modified=None, change_frequency=None, ne

In [27]:
for page in tree.sub_sitemaps[0].all_pages():
    print(page.url)

https://www.bvkap.de/
https://www.bvkap.de/politik.html
https://www.bvkap.de/unsere-positionen.html
https://www.bvkap.de/unsere-positionen/mittelstand.html
https://www.bvkap.de/unsere-positionen/nachhaltigkeit.html
https://www.bvkap.de/unsere-positionen/start-ups.html
https://www.bvkap.de/unsere-positionen/steuern.html
https://www.bvkap.de/unsere-positionen/ipo.html
https://www.bvkap.de/foerderprogramme.html
https://www.bvkap.de/beteiligungskapital/beteiligungskapital.html
https://www.bvkap.de/beteiligungskapital/finanzierungsanlaesse.html
https://www.bvkap.de/beteiligungskapital/kapitalsuche.html
https://www.bvkap.de/research/statistik-deutschland.html
https://www.bvkap.de/research/statistik-europa.html
https://www.bvkap.de/statistik/investorenbarometer.html
https://www.bvkap.de/presse-medien/studien.html
https://www.bvkap.de/presse-medien/studien/bvk-studien.html
https://www.bvkap.de/presse-medien/studien/weitere-studien.html
https://www.bvkap.de/presse-medien.html
https://www.bvkap.

In [22]:
tree.all_pages()

<generator object AbstractIndexSitemap.all_pages at 0x1134b35a0>

In [18]:
filter = ["mitglieder-details"]
pages = [page.url for page in tree.all_pages() if any(word.lower() in page.url.lower() for word in filter)]


In [19]:
len(pages)

2

In [15]:
pages

['https://www.bvkap.de/der-bvk/mitglieder/mitglieder-details.html',
 'https://www.bvkap.de/the-bvk/members/mitglieder-details.html']

In [42]:
with open("data/website/capital_requirements/100_Kapitalsuche I Bundesverband Beteiligungskapital.html") as fp:
    soup = BeautifulSoup(fp, 'html.parser')


In [51]:
import glob

In [62]:
urls = []

prefixes = ['capital_requirements', 'transaction_volume', 'turnover']

for prefix in prefixes:
    for path in glob.glob('data/website/{prefix}/*.html'.format(prefix=prefix)):
        with open(path, 'r') as fp:
            soup = BeautifulSoup(fp, 'html.parser')
            for el in soup.find_all('a', {'class': 'btn btn-danger'}):
                urls.append(el['href'])

In [63]:
len(urls)

1767

In [244]:
url = list(set(urls))[:3]
url

['https://www.bvkap.de/the-bvk/members/mitglieder-details/user/baincapital-private-equity-beteiligungsberatung-gmbh.html',
 'https://www.bvkap.de/the-bvk/members/mitglieder-details/user/naxicap-germany-gmbh.html',
 'https://www.bvkap.de/the-bvk/members/mitglieder-details/user/s-kap-unternehmensbeteiligungs-gmbh-co-kg.html']

In [312]:
firms = []

for url in set(urls):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    text = soup.get_text()

    firm = dict()

    firm['url'] = url

    try:
        name = soup.find_all('div', {'class': 'company-name'})[0].text.strip()
        firm['firm_name'] = name
        print(name)
    except:
        print('Could not parse name')
    
    try:
        address = ' '.join([x.text for x in soup.find_all('div', {'class': 'company-position'})[0].find_all('span')])
        firm['firm_address'] = address
    except:
        print('Could not parse address')

    try:
        profile = '\n'.join([x.text for x in soup.find_all('div', {'class': 'company-profil'})[0].find_all('p')])
        firm['firm_profile'] = profile
    except:
        print('Could not parse profile')

    try:
        cs = soup.find_all('div', {'id': 'case-collapse'})[0]
        header = cs.find_all('h2')[0].text
        text = '\n'.join([x.text for x in cs.find_all('p')])
        case_study = header + text
        firm['firm_case_study'] = case_study
    except:
        print('Could not parse case study')

    try:
        tab = soup.find_all('table', {'class': 'table table-striped'})[0]

        parsed_rows = []
        for row in tab.find_all('tr'):
            values = [x.get_text(separator='\n').strip() for x in row.find_all('td')]
            parsed_rows.append(values)
        firm['firm_info'] = parsed_rows
    except:
        print('Could not parse firm info')

    firms.append(firm)

BainCapital Private Equity Beteiligungsberatung GmbH
Could not parse profile
Could not parse case study
Naxicap Germany GmbH
Could not parse case study
S-Kap Unternehmensbeteiligungs GmbH & Co. KG
Could not parse case study
SIB Innovations- und Beteiligungsgesellschaft mbH
Could not parse case study
Mindful Capital Partners
Could not parse case study
CBG Commerz Beteiligungskapital GmbH & Co. KG
Could not parse case study
LEA Partners GmbH
Could not parse profile
Could not parse case study
Quadriga Capital Eigenkapitalberatung GmbH
Could not parse case study
MBMV Mittelständische Beteiligungsgesellschaft Mecklenburg-Vorpommern mbH
Could not parse profile
Could not parse case study
Bayern Kapital GmbH
Could not parse case study
Main Capital Partners
Could not parse case study
BC Partners
Could not parse case study
SANEMUS AG
Could not parse case study
LBBW Venture Capital GmbH
Could not parse case study
Unternehmertum Venture Capital Partners GmbH
Could not parse case study
btov Partner

In [461]:
with open('data/firm_data.json', 'w') as fjson:
    json.dump(firms, fjson)

In [45]:
with open('data/firm_data.json', 'r') as fjson:
    firms = json.load(fjson)

In [46]:
dfs = []

for firm in firms:
    if 'firm_info' in firm.keys():
        info_df = pd.DataFrame(firm['firm_info'])
        info_df.columns = ['key', 'value']
        if 'firm_name' in firm.keys():
            info_df.index=[firm['firm_name']]*len(info_df)
        else:
            info_df.index=[firm['url']]*len(info_df)
        info_df = info_df.pivot(columns='key', values='value')
        dfs.append(info_df)

info_df = pd.concat(dfs)

In [48]:
info_df.to_csv('bvkap_data.csv')

In [5]:
multipliers = {
    'm': 10000000,
    'Mio': 1000000
}

In [6]:
def convert_multiples(x):
    if not pd.isnull(x):
        price = Price.fromstring(x)
        for mult in multipliers.keys():
            if mult in x:
                amount = price.amount * multipliers[mult]
                return amount
    else:
        return None

def split_range(x):
    if not pd.isnull(x):
        if '-' in x:
            return [convert_multiples(p) for p in x.split('-')]
        elif '<' or '>' in x:
            return np.nan, convert_multiples(x)
    else:
        return np.nan, np.nan

In [7]:
info_df[['Minimum Equity / Investment Range', 'Maximum Equity / Investment Range']] = pd.DataFrame(info_df['Equity / Investment range'].apply(lambda x: split_range(x)))['Equity / Investment range'].tolist()
info_df[['Minimum turnover of companies', 'Maximum turnover of companies']] = pd.DataFrame(info_df['Turnover of those companies'].apply(lambda x: split_range(x)))['Turnover of those companies'].tolist()
info_df[['Minimum transaction volume', 'Maximum transaction volume']] = pd.DataFrame(info_df['Transaction volume'].apply(lambda x: split_range(x)))['Transaction volume'].tolist()
info_df['Capital'] = info_df['Capital'].apply(lambda x: convert_multiples(x))

info_df.drop(['Equity / Investment range', 'Turnover of those companies', 'Transaction volume'], axis=1, inplace=True)

numeric_cols = ['Minimum Equity / Investment Range', 'Maximum Equity / Investment Range',
                'Minimum turnover of companies', 'Maximum turnover of companies',
                'Minimum transaction volume', 'Maximum transaction volume', 'Capital']

for col in numeric_cols:
    info_df[col] = pd.to_numeric(info_df[col])

In [8]:
def bq_upload(df, schema, table_id):
    # Construct a BigQuery client object.
    client = bigquery.Client()
   
    job_config = bigquery.LoadJobConfig(
        # Specify a (partial) schema. All columns are always written to the
        # table. The schema is used to assist in data type definitions.
        schema=schema,
        # Optionally, set the write disposition. BigQuery appends loaded rows
        # to an existing table by default, but with WRITE_TRUNCATE write
        # disposition it replaces the table with the loaded data.
        write_disposition="WRITE_TRUNCATE",
    )

    job = client.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.

    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
    )

    

In [9]:
info_df.reset_index(inplace=True)
info_df.rename({'index': 'Firm Name'}, axis=1, inplace=True)
"""
for col in info_df.columns:
    clean_col = re.sub(r'[^a-zA-Z0-9ßäöüAÄÖÜ ]', '', col)
    clean_col = re.sub(r'\s+', '', clean_col)
    info_df.rename({col: clean_col}, axis=1, inplace=True)
"""

"\nfor col in info_df.columns:\n    clean_col = re.sub(r'[^a-zA-Z0-9ßäöüAÄÖÜ ]', '', col)\n    clean_col = re.sub(r'\\s+', '', clean_col)\n    info_df.rename({col: clean_col}, axis=1, inplace=True)\n"

In [10]:
schema = []

for col in info_df.columns[info_df.dtypes == 'object']:
    clean_col = re.sub(r'[^a-zA-Z0-9ßäöüAÄÖÜ ]', '', col)
    clean_col = re.sub(r'\s+', '', clean_col)
    info_df.rename({col: clean_col}, axis=1, inplace=True)
    schema.append(bigquery.SchemaField(name=clean_col, field_type=bigquery.enums.SqlTypeNames.STRING, description=col),)

for col in info_df.columns[info_df.dtypes != 'object']:
    clean_col = re.sub(r'[^a-zA-Z0-9ßäöüAÄÖÜ ]', '', col)
    clean_col = re.sub(r'\s+', '', clean_col)
    info_df.rename({col: clean_col}, axis=1, inplace=True)
    schema.append(bigquery.SchemaField(name=clean_col, field_type=bigquery.enums.SqlTypeNames.FLOAT, description=col),)

In [12]:
bq_upload(info_df, schema, "dastex-genai.pe_firm_scraped.firm_data")

Loaded 161 rows and 30 columns to dastex-genai.pe_firm_scraped.firm_data


In [14]:
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms import VertexAI
from langchain.chat_models import ChatVertexAI
from langchain.agents import create_sql_agent

In [2]:
service_account_file = "/Users/mb/.config/gcloud/application_default_credentials.json" # Change to where your service account key file is located
project = "dastex-genai"
dataset = "pe_firm_scraped"
table = "firm_data"
sqlalchemy_url = f'bigquery://{project}/{dataset}'

In [3]:
sqlalchemy_url

'bigquery://dastex-genai/pe_firm_scraped'

In [27]:
db = SQLDatabase.from_uri(sqlalchemy_url, custom_table_info=custom_table_info)
llm = VertexAI(
            model_name="text-bison@001",
            max_output_tokens=1024,
            temperature=0.1,
#            top_p=0.8,
#            top_k=40,
        )

chat_llm = ChatVertexAI(
                    model_name="chat-bison@001",
                    temperature=0.1,
                    max_output_tokens=1024,
                    top_p=0.8,
                    top_k=40,
                )

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(
llm=llm,
toolkit=toolkit,
verbose=True,
top_k=1000,
)

In [28]:
result = agent_executor.run("Which firms have offices in München?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: firm_data
Thought:I should query the schema of the firm_data table to see what columns I can use in my query.
Action: sql_db_schema
Action Input: firm_data
Observation: 
CREATE TABLE `firm_data` (
	`FirmName` STRING OPTIONS(description='Firm Name'), 
	`EquityType` STRING OPTIONS(description='Equity Type'), 
	`GeographicalFocus` STRING OPTIONS(description='Geographical Focus'), 
	`Industrysector` STRING OPTIONS(description='Industry sector'), 
	`InvestmentReasons` STRING OPTIONS(description='Investment Reasons'), 
	`InvestmentType` STRING OPTIONS(description='Investment Type'), 
	`InvestorTypes` STRING OPTIONS(description='Investor Types'), 
	`ManagingDirectorsPartners` STRING OPTIONS(description='Managing Directors / Partners'), 
	`Offices` STRING OPTIONS(description='Offices'), 
	`Shareholders` STRING OPTIONS(description='Shareholders'), 
	`ActivesinceinGermany` STRING OPTIONS(description='A

In [25]:
result

'The following firms have offices in München: creditshelf AG, EQT Partners GmbH, btov Partners, BainCapital Private Equity Beteiligungsberatung GmbH, Waterland Private Equity GmbH, /// HCM Handwerk Capital Management GmbH & Co. KG, Earlybird Venture Capital.'

In [23]:
agent_executor.run("Which firms have offices in München?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: firm_data
Thought:I should query the schema of the firm_data table to see what columns I can use in my query.
Action: sql_db_schema
Action Input: firm_data
Observation: 
CREATE TABLE `firm_data` (
	`FirmName` STRING OPTIONS(description='Firm Name'), 
	`EquityType` STRING OPTIONS(description='Equity Type'), 
	`GeographicalFocus` STRING OPTIONS(description='Geographical Focus'), 
	`Industrysector` STRING OPTIONS(description='Industry sector'), 
	`InvestmentReasons` STRING OPTIONS(description='Investment Reasons'), 
	`InvestmentType` STRING OPTIONS(description='Investment Type'), 
	`InvestorTypes` STRING OPTIONS(description='Investor Types'), 
	`ManagingDirectorsPartners` STRING OPTIONS(description='Managing Directors / Partners'), 
	`Offices` STRING OPTIONS(description='Offices'), 
	`Shareholders` STRING OPTIONS(description='Shareholders'), 
	`ActivesinceinGermany` STRING OPTIONS(description='A

'The firms that have offices in München are: creditshelf AG, EQT Partners GmbH, btov Partners, BainCapital Private Equity Beteiligungsberatung GmbH, Waterland Private Equity GmbH, /// HCM Handwerk Capital Management GmbH & Co. KG, and Earlybird Venture Capital.'

In [515]:
few_shots = {'Which firms have offices in Munich?': 'SELECT DISTINCT FirmName FROM firm_data WHERE Offices LIKE "%Munich%"',
              'Which firms have offices in Paris and what is their Turnover?': 'SELECT DISTINCT FirmName, Maximumturnoverofcompanies FROM firm_data WHERE Offices LIKE "%Paris%";',
              "List all tracks in the 'Rock' genre.": "SELECT * FROM tracks WHERE GenreId = (SELECT GenreId FROM genres WHERE Name = 'Rock');",
              'Find the total duration of all tracks.': 'SELECT SUM(Milliseconds) FROM tracks;',
              'List all customers from Canada.': "SELECT * FROM customers WHERE Country = 'Canada';",
              'How many tracks are there in the album with ID 5?': 'SELECT COUNT(*) FROM tracks WHERE AlbumId = 5;',
              'Find the total number of invoices.': 'SELECT COUNT(*) FROM invoices;',
              'List all tracks that are longer than 5 minutes.': 'SELECT * FROM tracks WHERE Milliseconds > 300000;',
              'Who are the top 5 customers by total purchase?': 'SELECT CustomerId, SUM(Total) AS TotalPurchase FROM invoices GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;',
              'Which albums are from the year 2000?': "SELECT * FROM albums WHERE strftime('%Y', ReleaseDate) = '2000';",
              'How many employees are there': 'SELECT COUNT(*) FROM "employee"'
             }

In [516]:
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001")

few_shot_docs = [Document(page_content=question, metadata={'sql_query': few_shots[question]}) for question in few_shots.keys()]
vector_db = FAISS.from_documents(few_shot_docs, embeddings)
retriever = vector_db.as_retriever()

In [517]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool_description = """
This tool will help you understand similar examples to adapt them to the user question.
Input to this tool should be the user question.
"""

retriever_tool = create_retriever_tool(
        retriever,
        name='sql_get_similar_examples',
        description=tool_description
    )
custom_tool_list = [retriever_tool]

In [523]:
from langchain.agents import create_sql_agent, AgentType
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.utilities import SQLDatabase
from langchain.chat_models import ChatOpenAI

db = SQLDatabase.from_uri(sqlalchemy_url)
llm = ChatOpenAI(model_name='gpt-4',temperature=0, openai_api_key=os.environ['OPENAI_API_KEY'])

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

custom_suffix = """
I should first get the similar examples I know.
If the examples are enough to construct the query, I can build it.
Otherwise, I can then look at the tables in the database to see what I can query.
Then I should query the schema of the most relevant tables.
Format the output in currency format with million, billion and trillion denomination.
"""

agent = create_sql_agent(llm=llm,
                         toolkit=toolkit,
                         verbose=True,
                         agent_type=AgentType.OPENAI_FUNCTIONS,
                         extra_tools=custom_tool_list,
                         suffix=custom_suffix
                        )

In [524]:
agent.run("Which firms have offices in Munich and what is their capital?")



> Entering new AgentExecutor chain...

Invoking: `sql_get_similar_examples` with `Which firms have offices in Munich and what is their capital?`


[Document(page_content='Which firms have offices in Munich?', metadata={'sql_query': 'SELECT DISTINCT FirmName FROM firm_data WHERE Offices LIKE "%Munich%"'}), Document(page_content='Which firms have offices in Paris and what is their Turnover?', metadata={'sql_query': 'SELECT DISTINCT FirmName, Maximumturnoverofcompanies FROM firm_data WHERE Offices LIKE "%Paris%";'}), Document(page_content='How many employees are there', metadata={'sql_query': 'SELECT COUNT(*) FROM "employee"'}), Document(page_content='List all tracks that are longer than 5 minutes.', metadata={'sql_query': 'SELECT * FROM tracks WHERE Milliseconds > 300000;'})]
Invoking: `sql_db_schema` with `firm_data`
responded: {content}


CREATE TABLE `firm_data` (
	`FirmName` STRING OPTIONS(description='Firm Name'), 
	`EquityType` STRING OPTIONS(description='Equity Type'), 
	`Geogra

'The firms that have offices in Munich and their capital are:\n\n1. SwanCap Partners GmbH with a capital of 30,000,000,000\n2. Capnamic Ventures Management GmbH with a capital of 3,130,000,000'

In [77]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool_description = """
This tool will help you understand similar examples to the users question. If the user asks 
about investment firms then you should use this tool.
This tool will help you understand similar examples to adapt them to the user question.
Input to this tool should be the user question.
"""

retriever_tool = create_retriever_tool(
        retriever,
        name='sql_get_similar_examples',
        description=tool_description
    )
custom_tool_list = [retriever_tool]

In [398]:
custom_suffix = """
I should first get the similar examples I know.
If the examples are enough to construct the query, I can build it.
Otherwise, I can then look at the tables in the database to see what I can query.
Then I should query the schema of the most relevant tables
{agent_scratchpad}
{input}
"""

llm = VertexAI(
            model_name="text-bison@001",
            max_output_tokens=1024,
            temperature=0.1,
#            top_p=0.8,
#            top_k=40,
        )

chat_llm = ChatVertexAI(
                    model_name="chat-bison@001",
                    temperature=0.1,
                    max_output_tokens=1024,
                    top_p=0.8,
                    top_k=40,
                )

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

finst = """
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question. You should include all the information returned, and include it in a bullet point format. 
"""

agent = create_sql_agent(llm=chat_llm,
                         toolkit=toolkit,
                         verbose=True,
                         agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         extra_tools=custom_tool_list,
                         format_instructions=finst
                        )

In [400]:
agent_executor.run("Which firms have offices in Munich? What is their capital?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: firm_data
Thought:I should query the schema of the firm_data table to see what columns are available.
Action: sql_db_schema
Action Input: firm_data
Observation: 
CREATE TABLE `firm_data` (
	`FirmName` STRING OPTIONS(description='Firm Name'), 
	`EquityType` STRING OPTIONS(description='Equity Type'), 
	`GeographicalFocus` STRING OPTIONS(description='Geographical Focus'), 
	`Industrysector` STRING OPTIONS(description='Industry sector'), 
	`InvestmentReasons` STRING OPTIONS(description='Investment Reasons'), 
	`InvestmentType` STRING OPTIONS(description='Investment Type'), 
	`InvestorTypes` STRING OPTIONS(description='Investor Types'), 
	`ManagingDirectorsPartners` STRING OPTIONS(description='Managing Directors / Partners'), 
	`Offices` STRING OPTIONS(description='Offices'), 
	`Shareholders` STRING OPTIONS(description='Shareholders'), 
	`ActivesinceinGermany` STRING OPTIONS(description='Active si

'The firms that have offices in Munich are Capnamic Ventures Management GmbH and SwanCap Partners GmbH. Their capital is 3130000000.0 and 30000000000.0 respectively.'

In [150]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [151]:
import logging
logging.getLogger('langchain_experimental.sql').setLevel(logging.INFO)
logging.getLogger('langchain.utilities').setLevel(logging.INFO)

In [395]:
print('Use the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question')

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


In [152]:
from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

If someone asks for the table foobar, they really mean the employee table.

Question: {input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

In [208]:
custom_table_info = {
    "firm_data": """
CREATE TABLE `firm_data` (
	`FirmName` STRING OPTIONS(description='Firm Name'), 
	`EquityType` STRING OPTIONS(description='Equity Type'), 
	`GeographicalFocus` STRING OPTIONS(description='Geographical Focus'), 
	`Industrysector` STRING OPTIONS(description='Industry sector'), 
	`InvestmentReasons` STRING OPTIONS(description='Investment Reasons'), 
	`InvestmentType` STRING OPTIONS(description='Investment Type'), 
	`InvestorTypes` STRING OPTIONS(description='Investor Types'), 
	`ManagingDirectorsPartners` STRING OPTIONS(description='Managing Directors / Partners'), 
	`Offices` STRING OPTIONS(description='Offices'), 
	`Shareholders` STRING OPTIONS(description='Shareholders'), 
	`ActivesinceinGermany` STRING OPTIONS(description='Active since (in Germany)'), 
	`Capital` FLOAT64 OPTIONS(description='Capital'), 
	`Capitalyear` STRING OPTIONS(description='Capital year'), 
	`FundCount` STRING OPTIONS(description='Fund Count'), 
	`Photo` STRING OPTIONS(description='Photo'), 
	`PortfolioCompanies` STRING OPTIONS(description='Portfolio Companies'), 
	`Professionals` STRING OPTIONS(description='Professionals'), 
	`Equityholdings` STRING OPTIONS(description='Equity holdings'), 
	`Equityholdingsyear` STRING OPTIONS(description='Equity holdings year'), 
	`FundYear` STRING OPTIONS(description='Fund Year'), 
	`Investmentcases` STRING OPTIONS(description='Investment cases'), 
	`Investors` STRING OPTIONS(description='Investors'), 
	`PortfoliocompanyYear` STRING OPTIONS(description='Portfolio company Year'), 
	`Staffcount` STRING OPTIONS(description='Staff count'), 
	`MinimumEquityInvestmentRange` FLOAT64 OPTIONS(description='Minimum Equity / Investment Range'), 
	`MaximumEquityInvestmentRange` FLOAT64 OPTIONS(description='Maximum Equity / Investment Range'), 
	`Minimumturnoverofcompanies` FLOAT64 OPTIONS(description='Minimum turnover of companies'), 
	`Maximumturnoverofcompanies` FLOAT64 OPTIONS(description='Maximum turnover of companies'), 
	`Minimumtransactionvolume` FLOAT64 OPTIONS(description='Minimum transaction volume'), 
	`Maximumtransactionvolume` FLOAT64 OPTIONS(description='Maximum transaction volume')
)
    """
}

In [576]:
from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

The function REGEXP does not exist in BigQuery. Instead, use LIKE.

You should include all the information returned, and include it in a bullet point format. 
Do not make up numbers, include only numbers that are returned from the query.

Question: {input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

In [535]:
"""
llm = VertexAI(
            model_name="text-bison@001",
            max_output_tokens=1024,
            temperature=0.1,
            #top_p=0.8,
            #top_k=40,
        )
"""

llm = ChatOpenAI(model_name='gpt-4',temperature=0, openai_api_key=os.environ['OPENAI_API_KEY'])


db = SQLDatabase.from_uri(sqlalchemy_url, custom_table_info=custom_table_info, include_tables=['firm_data'])
db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT, verbose=True, use_query_checker=True,  return_intermediate_steps=False, top_k=20)

In [544]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(input_key='input', memory_key="history")
dbchain = SQLDatabaseChain(
        llm_chain=LLMChain(llm=llm, prompt=PROMPT, memory=memory),
        database=db, 
        verbose=True
    )

In [595]:
from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

The database contains a lot of data, so it is advised to use DISTINCT operators in order to reduce the amount of data. Please use the DISTINCT column operator
where possible.
As the database contains some erroneuous characters, it is better to use LIKE instead of = when filtering.
The function REGEXP does not exist in BigQuery. Instead, use LIKE.

You should include all the information returned, and include it in a bullet point format. 
Do not make up numbers, include only numbers that are returned from the query.
Format any decimal numbers as currency figures.

Question: {input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

In [605]:
"""
https://github.com/langchain-ai/langchain/issues/6918
"""
from langchain import SQLDatabase
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", openai_api_key=os.environ['OPENAI_API_KEY']) # type: ignore
db = SQLDatabase.from_uri(sqlalchemy_url)
db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT, verbose=True)
tools = [
    Tool(
        name="dbchain",
        func=db_chain.run,
        description="Chat with SQLDB",
    )
]

agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True, 
    agent_kwargs=agent_kwargs, 
    memory=memory
)

In [606]:
result = agent({'input': 'Hello'})



> Entering new AgentExecutor chain...
Hi there! How can I assist you today?

> Finished chain.


In [609]:
result = agent({'input': 'How many firms are in the database?'})



> Entering new AgentExecutor chain...

Invoking: `dbchain` with `SELECT COUNT(*) FROM firms`




> Entering new SQLDatabaseChain chain...
SELECT COUNT(*) FROM firms
SQLQuery:SELECT COUNT(*) FROM firm_data
SQLResult: [(161,)]
Answer:Final answer here: There are 161 rows in the firm_data table.
> Finished chain.
Final answer here: There are 161 rows in the firm_data table.There are 161 firms in the database.

> Finished chain.


In [588]:
result = agent({'input': 'Which firms have offices in Paris, and what capital do they have?'})



> Entering new AgentExecutor chain...

Invoking: `dbchain` with `SELECT firm_name, capital FROM firms WHERE office_location = 'Paris'`




> Entering new SQLDatabaseChain chain...
SELECT firm_name, capital FROM firms WHERE office_location = 'Paris'
SQLQuery:SELECT FirmName, Capital FROM firm_data WHERE Offices LIKE '%Paris%'
SQLResult: [('Permira Beteiligungsberatung GmbH', None), ('Bridgepoint GmbH', None), ('Argos Wityu GmbH', 14000000000.0), ('3i Deutschland Gesellschaft für Industriebeteiligungen mbH', 160000000000.0), ('https://www.bvkap.de/the-bvk/members/mitglieder-details/user/africinvest.html', None), ('ARDIAN Germany GmbH', 1500000000000.0), ('Naxicap Germany GmbH', 40000000000.0), ('BC Partners', None), ('Demeter Investment Managers SA', 13000000000.0), ('H.I.G. European Capital Partners GmbH', None), ('Advent International GmbH', None), ('Gimv', 14000000000.0)]
Answer:- Permira Beteiligungsberatung GmbH: No capital information available
- Bridgepoint GmbH: No capital info

In [589]:
print(result['output'])

The following firms have offices in Paris and their respective capitals:

1. Permira Beteiligungsberatung GmbH: No capital information available
2. Bridgepoint GmbH: No capital information available
3. Argos Wityu GmbH: Capital = 14,000,000,000.0
4. 3i Deutschland Gesellschaft für Industriebeteiligungen mbH: Capital = 160,000,000,000.0
5. https://www.bvkap.de/the-bvk/members/mitglieder-details/user/africinvest.html: No capital information available
6. ARDIAN Germany GmbH: Capital = 1,500,000,000,000.0
7. Naxicap Germany GmbH: Capital = 40,000,000,000.0
8. BC Partners: No capital information available
9. Demeter Investment Managers SA: Capital = 13,000,000,000.0
10. H.I.G. European Capital Partners GmbH: No capital information available
11. Advent International GmbH: No capital information available
12. Gimv: Capital = 14,000,000,000.0

Please note that capital information may not be available for some firms.


In [592]:
result = agent({'input': 'Which firms have a geographical focus on Kanada?'})



> Entering new AgentExecutor chain...

Invoking: `dbchain` with `SELECT name FROM firms WHERE geographical_focus LIKE '%Kanada%'`




> Entering new SQLDatabaseChain chain...
SELECT name FROM firms WHERE geographical_focus LIKE '%Kanada%'
SQLQuery:SELECT FirmName 
FROM firm_data 
WHERE GeographicalFocus LIKE '%Kanada%'
SQLResult: [('TVM Capital Life Science',), ('Yttrium GmbH (vormals Digitalplus GmbH)',), ('Evonik Venture Capital GmbH',), ('SwanCap Partners GmbH',), ('Crédit Mutuel Equity',), ('SGT German Private Equity GmbH & Co KGaA',), ('Signature Ventures',)]
Answer:- TVM Capital Life Science
- Yttrium GmbH (vormals Digitalplus GmbH)
- Evonik Venture Capital GmbH
- SwanCap Partners GmbH
- Crédit Mutuel Equity
- SGT German Private Equity GmbH & Co KGaA
- Signature Ventures
> Finished chain.
- TVM Capital Life Science
- Yttrium GmbH (vormals Digitalplus GmbH)
- Evonik Venture Capital GmbH
- SwanCap Partners GmbH
- Crédit Mutuel Equity
- SGT German Private Equity GmbH & Co KGaA
- S

In [593]:
print(result['output'])

The following firms have a geographical focus on Canada:

1. TVM Capital Life Science
2. Yttrium GmbH (formerly Digitalplus GmbH)
3. Evonik Venture Capital GmbH
4. SwanCap Partners GmbH
5. Crédit Mutuel Equity
6. SGT German Private Equity GmbH & Co KGaA
7. Signature Ventures


In [550]:
result = db_chain({'query': 'Hello. How are you?'})



> Entering new SQLDatabaseChain chain...
Hello. How are you?
SQLQuery:N/A (No query needed for this question)

DatabaseError: (google.cloud.bigquery.dbapi.exceptions.DatabaseError) 400 Syntax error: Unexpected identifier "N" at [1:1]

Location: europe-west3
Job ID: 47afe51d-93c0-4fbb-bd0b-381fdab031c7

[SQL: N/A (No query needed for this question)]
(Background on this error at: https://sqlalche.me/e/14/4xp6)

In [545]:
result = db_chain.run("What firms have an office in München, what is their capital?")



> Entering new SQLDatabaseChain chain...
What firms have an office in München, what is their capital?
SQLQuery:SELECT FirmName, Capital
FROM `firm_data`
WHERE Offices LIKE '%München%'
SQLResult: [('creditshelf AG', None), ('EQT Partners GmbH', None), ('btov Partners', 4200000000.0), ('BainCapital Private Equity Beteiligungsberatung GmbH', None), ('Waterland Private Equity GmbH', None), ('/// HCM Handwerk Capital Management GmbH & Co. KG', 30000000.0), ('Earlybird Venture Capital', 8000000000.0)]
Answer:- creditshelf AG has no capital information available
- EQT Partners GmbH has no capital information available
- btov Partners has a capital of 4200000000.0
- BainCapital Private Equity Beteiligungsberatung GmbH has no capital information available
- Waterland Private Equity GmbH has no capital information available
- /// HCM Handwerk Capital Management GmbH & Co. KG has a capital of 30000000.0
- Earlybird Venture Capital has a capital of 8000000000.0
> Finished chain.


In [547]:
result

'- creditshelf AG has no capital information available\n- EQT Partners GmbH has no capital information available\n- btov Partners has a capital of 4200000000.0\n- BainCapital Private Equity Beteiligungsberatung GmbH has no capital information available\n- Waterland Private Equity GmbH has no capital information available\n- /// HCM Handwerk Capital Management GmbH & Co. KG has a capital of 30000000.0\n- Earlybird Venture Capital has a capital of 8000000000.0'

In [537]:
print(result)

- creditshelf AG has no capital information available
- EQT Partners GmbH has no capital information available
- btov Partners has a capital of 4200000000.0
- BainCapital Private Equity Beteiligungsberatung GmbH has no capital information available
- Waterland Private Equity GmbH has no capital information available
- /// HCM Handwerk Capital Management GmbH & Co. KG has a capital of 30000000.0
- Earlybird Venture Capital has a capital of 8000000000.0


In [538]:
import math

millnames = ['',' Thousand',' Million',' Billion',' Trillion']

def millify(n):
    n = float(n)
    millidx = max(0,min(len(millnames)-1,
                        int(math.floor(0 if n == 0 else math.log10(abs(n))/3))))

    return '{:.0f}{} €'.format(n / 10**(3 * millidx), millnames[millidx])

In [539]:
match = re.compile(r'([0-9]+\.\d+)+')
items = re.findall(match, result)


In [540]:
items

['4200000000.0', '30000000.0', '8000000000.0']

In [541]:
result

'- creditshelf AG has no capital information available\n- EQT Partners GmbH has no capital information available\n- btov Partners has a capital of 4200000000.0\n- BainCapital Private Equity Beteiligungsberatung GmbH has no capital information available\n- Waterland Private Equity GmbH has no capital information available\n- /// HCM Handwerk Capital Management GmbH & Co. KG has a capital of 30000000.0\n- Earlybird Venture Capital has a capital of 8000000000.0'

In [542]:
for item in items:
    result = result.replace(item, millify(item))

In [543]:
print(result)

- creditshelf AG has no capital information available
- EQT Partners GmbH has no capital information available
- btov Partners has a capital of 4 Billion €
- BainCapital Private Equity Beteiligungsberatung GmbH has no capital information available
- Waterland Private Equity GmbH has no capital information available
- /// HCM Handwerk Capital Management GmbH & Co. KG has a capital of 30 Million €
- Earlybird Venture Capital has a capital of 8 Billion €


In [447]:
30000000000.0 / 3130000000.0

9.584664536741213

In [220]:
prompt = PromptTemplate(

  input_variables=["Brand", "Product"],

  template="What would be the name of {Brand} that sells {Product}?",

)


'Capnamic Ventures Management GmbH, SwanCap Partners GmbH'

In [187]:
print(result['result'])

The firms with offices in München and their capital are: creditshelf AG, EQT Partners GmbH, btov Partners, BainCapital Private Equity Beteiligungsberatung GmbH, Waterland Private Equity GmbH, /// HCM Handwerk Capital Management GmbH & Co. KG, and Earlybird Venture Capital.


In [124]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [126]:
agent_executor.run("Which firms have offices in München and what is their minimum turnover?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: firm_data
Thought:I should query the schema of the firm_data table to see what columns I can use in my query.
Action: sql_db_schema
Action Input: firm_data
Observation: 
CREATE TABLE `firm_data` (
	`FirmName` STRING OPTIONS(description='Firm Name'), 
	`EquityType` STRING OPTIONS(description='Equity Type'), 
	`GeographicalFocus` STRING OPTIONS(description='Geographical Focus'), 
	`Industrysector` STRING OPTIONS(description='Industry sector'), 
	`InvestmentReasons` STRING OPTIONS(description='Investment Reasons'), 
	`InvestmentType` STRING OPTIONS(description='Investment Type'), 
	`InvestorTypes` STRING OPTIONS(description='Investor Types'), 
	`ManagingDirectorsPartners` STRING OPTIONS(description='Managing Directors / Partners'), 
	`Offices` STRING OPTIONS(description='Offices'), 
	`Shareholders` STRING OPTIONS(description='Shareholders'), 
	`ActivesinceinGermany` STRING OPTIONS(description='A

'The minimum turnover of the firms with offices in München is 1000000.0.'

In [16]:
from langchain.agents import Tool, LLMSingleActionAgent, AgentExecutor
from langchain.agents import AgentType, initialize_agent

In [525]:
def setup_knowledge_base():
    """
    We assume that the product knowledge base is simply a text file.
    """
    db = SQLDatabase.from_uri(sqlalchemy_url)
    llm = ChatOpenAI(model_name='gpt-3.5-turbo',temperature=0, openai_api_key=os.environ['OPENAI_API_KEY'])

    toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    top_k=1000,
    )
    db = SQLDatabase.from_uri(sqlalchemy_url, custom_table_info=custom_table_info, include_tables=['firm_data'])
    db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT, verbose=True, use_query_checker=True,  return_intermediate_steps=False, top_k=20)
    return db_chain

def get_tools():
    # query to get_tools can be used to be embedded and relevant tools found
    # see here: https://langchain-langchain.vercel.app/docs/use_cases/agents/custom_agent_with_plugin_retrieval#tool-retriever

    # we only use one tool for now, but this is highly extensible!
    knowledge_base = setup_knowledge_base()
    tools = [
        Tool(
            name="FirmSearch",
            func=knowledge_base.run,
            description="This tool can be used to find out information about private equity firms. It has comprehensive information regarding the firm names, address, turnover, capital requirements.",
        )
    ]

    return tools

In [526]:
knowledge_base = setup_knowledge_base()

In [527]:
result = knowledge_base("How many firms?")



> Entering new SQLDatabaseChain chain...
How many firms?
SQLQuery:SELECT COUNT(*) AS FirmCount
FROM `firm_data`
SQLResult: [(161,)]
Answer:Final answer: There are 161 firms.
> Finished chain.


In [528]:
result

{'query': 'How many firms?', 'result': 'Final answer: There are 161 firms.'}

In [529]:
llm = ChatOpenAI(model_name='gpt-4',temperature=0, openai_api_key=os.environ['OPENAI_API_KEY'])

agent = initialize_agent(
    llm=llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True, tools=get_tools(), handle_parsing_errors=True,
)

In [530]:
agent.run('Hello. How many firms are based in Munich and what is their capital?')



> Entering new AgentExecutor chain...

Invoking: `FirmSearch` with `Munich`




> Entering new SQLDatabaseChain chain...
Munich
SQLQuery:SELECT * FROM `firm_data` WHERE `Offices` LIKE '%Munich%'
SQLResult: [('Capnamic Ventures Management GmbH', 'Minderheitsbeteiligung\nDirekte / offene Beteiligungen', 'Deutschland\nEuropa', 'Finanzdienstleistungen\nHandel / E-Commerce\nInternet\nKommunikationstechnologie\nSoftware / IT', 'Seed-Finanzierung\nStart up-Finanzierung', 'Unabhängige Beteiligungsgesellschaft', 'Privatpersonen / Family Offices\nÖffentliche Hand (Bund, Länder, EU)\nKreditinstitute\nUnternehmen (Industrie, Dienstleistungen)', 'Jörg Binnenbrücker\nChristian Siegele\nOlaf Jacobi\nChristian Knott', 'Linienstraße 214\n10119 Berlin\nLudwigstraße 9\n80539 Munich', None, '2013', 3130000000.0, None, '3', None, '38', None, '€ 29 m', None, None, 'adjust GmbH\n\n\n\n\nLeanIX\n\n\n\n\nStaffbase\n\n\n\n\ngetsafe\n\n\n\n\nFATMAP\n\n\n\n\nUserlane\n\n\n\n\nParcellab\n\n\n\n\nNDGIT\n\n\n\n\nC

'There are two firms based in Munich:\n\n1. Capnamic Ventures Management GmbH\n   - Capital: €3,130,000,000\n   - Minimum Equity / Investment Range: €1,000,000\n   - Maximum Equity / Investment Range: €15,000,000\n\n2. SwanCap Partners GmbH\n   - Capital: €2,000\n   - Capital year: €30,000,000,000\n\nPlease note that the capital year for SwanCap Partners GmbH seems to be significantly larger than the capital, which might be a mistake in the data.'

In [203]:
def setup_knowledge_base():
    """
    We assume that the product knowledge base is simply a text file.
    """
    llm = VertexAI(
            model_name="text-bison@001",
            max_output_tokens=1024,
            temperature=0.1,
#            top_p=0.8,
#            top_k=40,
        )


    db = SQLDatabase.from_uri(sqlalchemy_url, custom_table_info=custom_table_info)
    db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker=True)
    return db_chain

def get_tools():
    # query to get_tools can be used to be embedded and relevant tools found
    # see here: https://langchain-langchain.vercel.app/docs/use_cases/agents/custom_agent_with_plugin_retrieval#tool-retriever

    # we only use one tool for now, but this is highly extensible!
    knowledge_base = setup_knowledge_base()
    tools = [
        Tool(
            name="FirmSearch",
            func=knowledge_base.run,
            description="This tool can be used to find out information about private equity firms. It has comprehensive information regarding the firm names, address, turnover, capital requirements.",
        )
    ]

    return tools

In [206]:
chat_llm = ChatVertexAI(
                    model_name="chat-bison@001",
                    temperature=0.1,
                    max_output_tokens=1024,
                    top_p=0.8,
                    top_k=40,
                )

agent = initialize_agent(
    llm=chat_llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True, tools=get_tools(), handle_parsing_errors=True,
)

In [207]:
agent.run('What is the transaction volume of firms based in Munich?')



> Entering new AgentExecutor chain...
Question: What is the transaction volume of firms based in Munich?
Thought: I need to find firms based in Munich.
Action:
```
{
  "action": "FirmSearch",
  "action_input": {
    "location": "Munich"
  }
}
```


> Entering new SQLDatabaseChain chain...
Munich
SQLQuery:SELECT FirmName FROM firm_data WHERE Offices LIKE '%Munich%' ORDER BY FirmName LIMIT 5
SQLResult: [('Capnamic Ventures Management GmbH',), ('SwanCap Partners GmbH',)]
Answer:Capnamic Ventures Management GmbH, SwanCap Partners GmbH
> Finished chain.

Observation: Capnamic Ventures Management GmbH, SwanCap Partners GmbH
Thought:Question: What is the transaction volume of firms based in Munich?
Thought: I need to find firms based in Munich.
Action:
```
{
  "action": "FirmSearch",
  "action_input": {
    "location": "Munich"
  }
}
```



> Entering new SQLDatabaseChain chain...
Munich
SQLQuery:SELECT FirmName, GeographicalFocus FROM firm_data WHERE GeographicalFocus LIKE '%Munich%' ORDER

'The transaction volume of firms based in Munich is 100 million euros.'